In [1]:
!pip install langchain_community tiktoken langchainhub chromadb langchain
!pip install --upgrade --quiet  langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 5.9 MB/s eta 0:00:00
 

In [2]:
import os
import getpass
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ["USER_AGENT"] = "GChat/1.0"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
google_api_key = getpass.getpass("Enter your Google API key: ")

Enter your LangSmith API key: ··········
Enter your Google API key: ··········


In [4]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
# Embed
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=GoogleGenerativeAIEmbeddings(model="models/text-embedding-004", google_api_key=google_api_key))

retriever = vectorstore.as_retriever()

In [5]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five
different versions of the given user question to retrieve relevant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

generate_queries = (
    prompt_perspectives
    | ChatGoogleGenerativeAI(model="models/gemini-1.5-flash", google_api_key=google_api_key, temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [6]:
generate_queries

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='You are an AI language model assistant. Your task is to generate five \ndifferent versions of the given user question to retrieve relevant documents from a vector \ndatabase. By generating multiple perspectives on the user question, your goal is to help\nthe user overcome some of the limitations of the distance-based similarity search. \nProvide these alternative questions separated by newlines. Original question: {question}'), additional_kwargs={})])
| ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7818a72beb00>, default_metadata=())
| StrOutputParser()
| RunnableLambda(...)

In [13]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})


In [17]:
from operator import itemgetter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash", google_api_key=google_api_key, temperature=0)

final_rag_chain = (
    {"context": retrieval_chain,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

response = final_rag_chain.invoke({"question":question})
#cleaned_output = " ".join(final_rag_chain.invoke({"question": question}).split())
#print(cleaned_output)

In [18]:
import re

# Split the text by periods followed by a space to create sentences
sentences = re.split(r'(?<=\.)\s+', response)

# Join sentences into structured format
structured_output = "\n\n".join(sentences)

print(structured_output)


Task decomposition is the process of breaking down a large, complex task into smaller, more manageable subtasks.

This allows LLM agents to handle complex tasks more efficiently.

The document mentions several methods for task decomposition:

* **Chain of Thought (CoT):**  The model is prompted to "think step by step" to break down tasks into smaller steps.

* **Tree of Thoughts:** This extends CoT by exploring multiple reasoning possibilities at each step, creating a tree structure of potential solutions.

* **LLM prompting:**  Simple prompts like "Steps for XYZ...

1." or "What are the subgoals for achieving XYZ?" can be used to guide the LLM in decomposing tasks.

* **Task-specific instructions:**  Instructions tailored to the specific task, such as "Write a story outline" for writing a novel, can be used to guide decomposition.

* **Human inputs:**  Humans can provide input to help the LLM decompose tasks.




In [19]:
# Split by period or other delimiters to identify key points
points = re.split(r'(?<=\.)\s+', response)

# Format as bullet points
formatted_response = "\n".join([f"- {point.strip()}" for point in points])

print(formatted_response)


- Task decomposition is the process of breaking down a large, complex task into smaller, more manageable subtasks.
- This allows LLM agents to handle complex tasks more efficiently.
- The document mentions several methods for task decomposition:

* **Chain of Thought (CoT):**  The model is prompted to "think step by step" to break down tasks into smaller steps.
- * **Tree of Thoughts:** This extends CoT by exploring multiple reasoning possibilities at each step, creating a tree structure of potential solutions.
- * **LLM prompting:**  Simple prompts like "Steps for XYZ...
- 1." or "What are the subgoals for achieving XYZ?" can be used to guide the LLM in decomposing tasks.
- * **Task-specific instructions:**  Instructions tailored to the specific task, such as "Write a story outline" for writing a novel, can be used to guide decomposition.
- * **Human inputs:**  Humans can provide input to help the LLM decompose tasks.
- 


In [20]:
# Add some markdown structure to the response
structured_markdown = f"# Task Decomposition for LLM Agents\n\n{response}"

print(structured_markdown)


# Task Decomposition for LLM Agents

Task decomposition is the process of breaking down a large, complex task into smaller, more manageable subtasks. This allows LLM agents to handle complex tasks more efficiently. 

The document mentions several methods for task decomposition:

* **Chain of Thought (CoT):**  The model is prompted to "think step by step" to break down tasks into smaller steps.
* **Tree of Thoughts:** This extends CoT by exploring multiple reasoning possibilities at each step, creating a tree structure of potential solutions.
* **LLM prompting:**  Simple prompts like "Steps for XYZ... 1." or "What are the subgoals for achieving XYZ?" can be used to guide the LLM in decomposing tasks.
* **Task-specific instructions:**  Instructions tailored to the specific task, such as "Write a story outline" for writing a novel, can be used to guide decomposition.
* **Human inputs:**  Humans can provide input to help the LLM decompose tasks. 



In [26]:
import json
import textwrap

# If response is a dictionary or list, pretty print it
try:
    parsed_response = json.loads(response)
    formatted_response = json.dumps(parsed_response, indent=4)
    # Apply word wrapping to each line
    wrapped_response = "\n".join([textwrap.fill(line, width=60) for line in formatted_response.splitlines()])

    print(formatted_response)
except:
    # If response is not a valid JSON, just wrap the text response
    wrapped_response = textwrap.fill(response, width=60)
    print(response)


Task decomposition is the process of breaking down a large, complex task into smaller, more manageable subtasks. This allows LLM agents to handle complex tasks more efficiently. 

The document mentions several methods for task decomposition:

* **Chain of Thought (CoT):**  The model is prompted to "think step by step" to break down tasks into smaller steps.
* **Tree of Thoughts:** This extends CoT by exploring multiple reasoning possibilities at each step, creating a tree structure of potential solutions.
* **LLM prompting:**  Simple prompts like "Steps for XYZ... 1." or "What are the subgoals for achieving XYZ?" can be used to guide the LLM in decomposing tasks.
* **Task-specific instructions:**  Instructions tailored to the specific task, such as "Write a story outline" for writing a novel, can be used to guide decomposition.
* **Human inputs:**  Humans can provide input to help the LLM decompose tasks. 

